# Getting Census data from CensusReporter
## Marketing use case for coffee from Mintel Report
### Homework for Digital Advertising, goal is to introduce APIs and use CensusReporter to determine what metro area has the highest population of our target market in launching a product 

In [11]:
#Neven Grgic
#Python Exercise 7
#10/11/18

import pandas as pd
import requests
import json

#data can be found in github repo
df = pd.read_csv('msas.csv')

In [12]:
#Select tables of interest from Census Reporter and make into one string to later paste into URL
tableids = ['B01001', 'B03002', 'B19001']
tableidstring = ','.join(tableids)

#Select cities of interest from csv and make into one string to later paste into URL
geoid = list(map(str,df['CBSA']))
for i in range(382):
    geoid[i] = "31000US" + geoid[i]
geoidstring = ','.join(geoid)    


In [13]:
#Get the json data from CensusReporter. Must be done in 2 sections as one request is too much for the webpage
requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring, geoidstring[0:2001])
loadedjson = requests.get(requesturl)
parsedjson = json.loads(loadedjson.text)

requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring, geoidstring[2002:])
loadedjson = requests.get(requesturl)
parsedjson2 = json.loads(loadedjson.text)


In [14]:
#get relevant income populations (50-75k) from each json and add to our df
IncomePop = []

for i in range(154):
    IncomePop.append(parsedjson['data'][geoid[i]]['B19001']['estimate']['B19001011']+
    parsedjson['data'][geoid[i]]['B19001']['estimate']['B19001012'])
for i in range(228):
    IncomePop.append(parsedjson2['data'][geoid[154+i]]['B19001']['estimate']['B19001011']+
    parsedjson2['data'][geoid[154+i]]['B19001']['estimate']['B19001012'])
    
df['Population Making 50-75k'] = IncomePop


In [15]:
#get relevant age range data (18-34) from each json and add to df
AgeRangePop = []

for i in range(154):
    AgeRangePop.append(parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001007'] +
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001008'] +
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001009']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001010']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001011']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001012']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001031']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001032']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001033']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001034']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001035']+
    parsedjson['data'][geoid[i]]['B01001']['estimate']['B01001036'])
for i in range(228):
    AgeRangePop.append(parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001007'] +
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001008'] +
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001009']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001010']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001011']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001012']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001031']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001032']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001033']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001034']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001035']+
    parsedjson2['data'][geoid[154+i]]['B01001']['estimate']['B01001036'])

df['Population aged 18-34'] = AgeRangePop


In [16]:
#get population of hispanics from each json and add to df
HispanicPop = []
for i in range(154):
    HispanicPop.append(parsedjson['data'][geoid[i]]['B03002']['estimate']['B03002012'])
for i in range(228):
    HispanicPop.append(parsedjson2['data'][geoid[154+i]]['B03002']['estimate']['B03002012'])

df['Hispanic Population'] = HispanicPop


In [17]:
#Find max population for income figure
print("========================================")
print("City with the Most People Making 50-75k")
print("========================================")
pd.set_option('display.max_columns', None)
HighIncome = df[df['Population Making 50-75k'] == max(df['Population Making 50-75k'])]
print(HighIncome[['NAME', 'Population Making 50-75k']])

#find max population for age range
print()
print("========================================")
print("City with the Most People Aged 18-34")
print("========================================")
TopAge = df[df['Population aged 18-34'] == max(df['Population aged 18-34'])]
print(TopAge[['NAME', 'Population aged 18-34']])

#find max population for Hispanic population
print()
print("========================================")
print("City with the Highest Hispanic Population")
print("========================================")
TopHispanic = df[df['Hispanic Population'] == max(df['Hispanic Population'])]
print(TopHispanic[['NAME', 'Hispanic Population']])

City with the Most People Making 50-75k
                                      NAME  Population Making 50-75k
249  New York-Newark-Jersey City, NY-NJ-PA                 1038188.0

City with the Most People Aged 18-34
                                      NAME  Population aged 18-34
249  New York-Newark-Jersey City, NY-NJ-PA              4806939.0

City with the Highest Hispanic Population
                                   NAME  Hispanic Population
210  Los Angeles-Long Beach-Anaheim, CA            6031492.0


In [18]:
#Rescale each parameter as a percentage and average across rows. Percentage works as an index essentially. 
df['Income %'] = df['Population Making 50-75k']/max(df['Population Making 50-75k'])
df['Age %'] = df['Population aged 18-34']/max(df['Population aged 18-34'])
df['Hispanic %'] = df['Hispanic Population']/max(df['Hispanic Population'])
df['Average %'] = df[['Income %', 'Age %', 'Hispanic %']].mean(axis = 1)

#Find the city with the highest average percentage to launch our product in
print()
print("========================================")
print("City with the Highest Average Percentage")
print("========================================")
TopPercentage = df[df['Average %'] == max(df['Average %'])]
print(TopPercentage[['NAME', 'Average %']])


City with the Highest Average Percentage
                                      NAME  Average %
249  New York-Newark-Jersey City, NY-NJ-PA   0.942684
